In [1]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import tensorflow as tf

In [2]:
# Read the databases from resources
health_factors = pd.read_csv("/content/Health_Risk_factors.csv")
freshwater = pd.read_csv("/content/freshwater.csv")
health_system=pd.read_csv("/content/health_system.csv")

In [3]:
health_system.columns

Index(['Country', 'Health expenditure Current % of GDP 2016',
       'Health expenditure Public % of current 2016',
       'Health expenditure Out of pocket % of current 2016',
       'Health expenditure per capita $ 2016',
       'Health expenditure per capita current PPP $ 2016',
       'External health expenditure (% of current health expenditure)   2016',
       'Health workers Physicians per 1,000 people 2009-18',
       'Health workers Nurses and midwives per 1,000 people 2009-18',
       'Specialist surgical workforce  per 100,000 population 2008-18',
       'Completeness of birth registration  % 2009-18',
       'Completeness of death registration  % 2008-16'],
      dtype='object')

In [4]:
merged_df = pd.concat([health_system, health_factors['Incidence of tuberculosis  per 100,000 people 2018']], axis=1)

In [5]:
merged_df.columns

Index(['Country', 'Health expenditure Current % of GDP 2016',
       'Health expenditure Public % of current 2016',
       'Health expenditure Out of pocket % of current 2016',
       'Health expenditure per capita $ 2016',
       'Health expenditure per capita current PPP $ 2016',
       'External health expenditure (% of current health expenditure)   2016',
       'Health workers Physicians per 1,000 people 2009-18',
       'Health workers Nurses and midwives per 1,000 people 2009-18',
       'Specialist surgical workforce  per 100,000 population 2008-18',
       'Completeness of birth registration  % 2009-18',
       'Completeness of death registration  % 2008-16',
       'Incidence of tuberculosis  per 100,000 people 2018'],
      dtype='object')

In [6]:
# Renaming columns in the health_factors DataFrame to more descriptive names
fw = merged_df.rename(columns={
    "Incidence of tuberculosis  per 100,000 people 2018": "tuberculosis",
    'External health expenditure (% of current health expenditure)   2016': "ext_health_expend",
    "Health expenditure Public % of current 2016": "public_health_expend",
    "Health workers Physicians per 1,000 people 2009-18": "physicians",
    "Health workers Nurses and midwives per 1,000 people 2009-18": "nurses",
    "Specialist surgical workforce  per 100,000 population 2008-18": "surgeries"
     })


In [8]:
df=fw[["tuberculosis","ext_health_expend","public_health_expend","physicians","nurses","surgeries"]]
df1 = df.replace('', np.nan)  # Replace empty values with NaN
df1 = df.dropna()  # Drop rows with any missing values

In [9]:
df1["tuberculosis"].describe()

count    146.000000
mean     114.315068
std      143.367036
min        0.000000
25%       13.250000
50%       50.000000
75%      159.750000
max      611.000000
Name: tuberculosis, dtype: float64

In [11]:
# Define the thresholds for classification
threshold_tuberculosis = 100
# Create a DataFrame to store the classified values
df2 = pd.DataFrame()
# Classify every columns
df2=df1[["ext_health_expend","public_health_expend","physicians","nurses", "surgeries"  ]]
df2['tuberculosis'] = [0 if i < threshold_tuberculosis else 1 for i in df1['tuberculosis']]
# Extract X and y
y = df2['tuberculosis'].values
X = df2.drop(columns='tuberculosis').values

In [12]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y,test_size=.2)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
l_1 = 50
l_2 = 50
input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=l_1, input_dim = input_features, activation = "tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=l_2, activation = "LeakyReLU"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                300       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,901
Trainable params: 2,901
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
4/4 [==============================] - 1s 6ms/step - loss: 0.6128 - accuracy: 0.6810
Epoch 2/50
4/4 [==============================] - 0s 4ms/step - loss: 0.5142 - accuracy: 0.7586
Epoch 3/50
4/4 [==============================] - 0s 4ms/step - loss: 0.4697 - accuracy: 0.7931
Epoch 4/50
4/4 [==============================] - 0s 4ms/step - loss: 0.4395 - accuracy: 0.7931
Epoch 5/50
4/4 [==============================] - 0s 4ms/step - loss: 0.4247 - accuracy: 0.7845
Epoch 6/50
4/4 [==============================] - 0s 6ms/step - loss: 0.4111 - accuracy: 0.8103
Epoch 7/50
4/4 [==============================] - 0s 4ms/step - loss: 0.4047 - accuracy: 0.8362
Epoch 8/50
4/4 [==============================] - 0s 4ms/step - loss: 0.3992 - accuracy: 0.8276
Epoch 9/50
4/4 [==============================] - 0s 5ms/step - loss: 0.3927 - accuracy: 0.8190
Epoch 10/50
4/4 [==============================] - 0s 4ms/step - loss: 0.3888 - accuracy: 0.8276
Epoch 11/50
4/4 [======================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.6392 - accuracy: 0.8333 - 158ms/epoch - 158ms/step
Loss: 0.6391882300376892, Accuracy: 0.8333333134651184


In [17]:
# Predict on the test data
y_pred = nn.predict(X_test)

# The predictions will be in the form of probabilities, so you might want to round them to get class labels (0 or 1)
y_pred_class = y_pred.round()
# Calculate the accuracy of the predictions
correct_predictions = (y_pred_class == y_test.reshape(-1, 1)).sum()
total_samples = len(y_test)
accuracy = correct_predictions / total_samples
print("Accuracy:", accuracy)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

1/1 [==============================] - 0s 89ms/step
Accuracy: 0.8333333333333334
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.85      0.87        20
           1       0.73      0.80      0.76        10

    accuracy                           0.83        30
   macro avg       0.81      0.82      0.82        30
weighted avg       0.84      0.83      0.84        30

